# Modelling a gallium arsenide surface

This example shows how to use the atomistic simulation environment or ASE for short,
to set up and run a particular calculation of a gallium arsenide surface.
ASE is a Python package to simplify the process of setting up,
running and analysing results from atomistic simulations across different simulation codes.
For more details on the integration DFTK provides with ASE,
see Atomistic simulation environment (ASE).

In this example we will consider modelling the (1, 1, 0) GaAs surface separated by vacuum.

Parameters of the calculation. Since this surface is far from easy to converge,
we made the problem simpler by choosing a smaller `Ecut` and smaller values
for `n_GaAs` and `n_vacuum`.
More interesting settings are `Ecut = 15` and `n_GaAs = n_vacuum = 20`.

In [1]:
miller = (1, 1, 0)   # Surface Miller indices
n_GaAs = 2           # Number of GaAs layers
n_vacuum = 4         # Number of vacuum layers
Ecut = 5             # Hartree
kgrid = (4, 4, 1);   # Monkhorst-Pack mesh

Use ASE to build the structure:

In [2]:
using ASEconvert
using PythonCall

a = 5.6537  # GaAs lattice parameter in Ångström (because ASE uses Å as length unit)
gaas = ase.build.bulk("GaAs", "zincblende"; a)
surface = ase.build.surface(gaas, miller, n_GaAs, 0, periodic=true);

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/kvmT8/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/L4cjh/CondaPkg.toml
    CondaPkg Resolving changes
             + ase
             + libstdcxx-ng
             + openssl
             + python
    CondaPkg Initialising pixi
             │ /home/runner/.julia/artifacts/cefba4912c2b400756d043a2563ef77a0088866b/bin/pixi
             │ init
             │ --format pixi
             └ /home/runner/work/DFTK.jl/DFTK.jl/docs/.CondaPkg
✔ Created /home/runner/work/DFTK.jl/DFTK.jl/docs/.CondaPkg/pixi.toml
    CondaPkg Wrote /home/runner/work/DFTK.jl/DFTK.jl/docs/.CondaPkg/pixi.toml
             │ [dependencies]
             │ openssl = ">=3, <3.6"
             │ libstdcxx-ng = ">=3.4,<15.0"
             │ ase = ">=3.23,<3.24"
             │ 
             │     [dependencies.python]
             │     channel = "conda-forge"
             │     build = "*cpython*"
             │     

Get the amount of vacuum in Ångström we need to add

In [3]:
d_vacuum = maximum(maximum, surface.cell) / n_GaAs * n_vacuum
surface = ase.build.surface(gaas, miller, n_GaAs, d_vacuum, periodic=true);

Write an image of the surface and embed it as a nice illustration:

In [4]:
ase.io.write("surface.png", surface * pytuple((3, 3, 1)), rotation="-90x, 30y, -75z")

Python: None

<img src="https://docs.dftk.org/stable/surface.png" width=500 height=500 />

Use the `pyconvert` function from `PythonCall` to convert the ASE atoms
to an AtomsBase-compatible system.
This can then be used in the same way as other `AtomsBase` systems
(see AtomsBase integration for details) to construct a DFTK model:

In [5]:
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.pbe.v0_1.largecore.gth")
model = model_DFT(pyconvert(AbstractSystem, surface);
                  functionals=PBE(),
                  temperature=1e-3,
                  smearing=DFTK.Smearing.Gaussian(),
                  pseudopotentials)

Model(gga_x_pbe+gga_c_pbe, 3D):
    lattice (in Bohr)    : [7.55469   , 0         , 0         ]
                           [0         , 7.55469   , 0         ]
                           [0         , 0         , 40.0648   ]
    unit cell volume     : 2286.6 Bohr³

    atoms                : As₂Ga₂
    pseudopot. family    : PseudoFamily("cp2k.nc.sr.pbe.v0_1.largecore.gth")

    num. electrons       : 16
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.Gaussian()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(gga_x_pbe, gga_c_pbe)
                           Entropy()

In the above we use the `pseudopotential` keyword argument to
assign the respective pseudopotentials to the imported `model.atoms`.
Try lowering the SCF convergence tolerance (`tol`)
or try `mixing=KerkerMixing()` to see the full challenge of this system.

In [6]:
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol=1e-6, mixing=LdosMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.58742666290                   -0.58    5.3    1.58s
  2   -16.72497540376       -0.86       -1.01    1.0    286ms
  3   -16.73053452007       -2.25       -1.57    2.1    269ms
  4   -16.73121718427       -3.17       -2.16    1.0    226ms
  5   -16.73132306905       -3.98       -2.59    2.0    251ms
  6   -16.73133350826       -4.98       -2.82    2.0    257ms
  7   -16.73081995680   +   -3.29       -2.45    2.1    268ms
  8   -16.73124756547       -3.37       -2.73    2.4    584ms
  9   -16.73113511542   +   -3.95       -2.64    2.0    232ms
 10   -16.73133898611       -3.69       -3.55    1.1    206ms
 11   -16.73133700064   +   -5.70       -3.49    3.1    289ms
 12   -16.73134005831       -5.51       -3.90    1.0    201ms
 13   -16.73133719440   +   -5.54       -3.55    2.2    259ms
 14   -16.73134019334       -5.52       -4.64    2.1    237ms
 15   -16

In [7]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             5.8593973 
    AtomicLocal         -105.6100139
    AtomicNonlocal      2.3494809 
    Ewald               35.5044300
    PspCorrection       0.2016043 
    Hartree             49.5614284
    Xc                  -4.5976637
    Entropy             -0.0000035

    total               -16.731340200436